In [14]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [26]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'technology+trend' # topic is "large language model"
#dataset = 'sp=3178' #Tech blog
#datasets = ['q=*[MarketSize]','sp=3178','sp=3814']

dataset = '&sp=3814' #industry news
# dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # dataset: venture capital investments
#dataset = 'q=*[FundingCompany|FundingProject|FundingOrganization]&site=www.sbir.gov|foerderportal.bund.de|cordis.europa.eu|www.nsf.gov|gtr.ukri.org'

#datasets = ['q=*[FundingCompany|FundingProject|FundingOrganization]&site=www.sbir.gov|foerderportal.bund.de|cordis.europa.eu|www.nsf.gov|gtr.ukri.org', '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572']
#combined_datasets = ''.join(datasets)  # Combine them into a single string
rows = str(5) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/fw/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text

Found 5 documents.


# Answering questions on the results with OpenAI GPT

In [30]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - You are focusing more in SIEMENS, it would be great to emphasize about SIEMENS's strength
        - Focus on Sense&Act.
        - Be creative but grounded
        - Avoid homogeneous ideas
        """

for doc in mergeflow_docs:
    user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```

    I wish to generate a business model canvas based on my finding and current trend, be detail as possible.
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
      )
    print(completion.choices[0].message.content)
    print("\n---\n")
    

Based on the information provided, let's create a business model canvas for a sports analytics solution:

Customer Segments:
1. Professional Sports Organizations: Targeting teams, leagues, and sports organizations that require data-driven insights to improve game strategies, player performance, and fan engagement.
2. Amateur Sports Organizations: Targeting smaller sports clubs and organizations that want to leverage analytics to enhance their performance and decision-making processes.

Value Propositions:
1. Data-Driven Decision-Making: Providing sports organizations with actionable insights derived from data analytics to optimize game strategies, player performance, and fan engagement.
2. Performance Analytics: Offering cutting-edge solutions such as video analytics and advanced wearable technology to enable in-depth performance analysis and health assessments.
3. Cloud-Based Deployment: Providing cost-effective, scalable, and efficient management of large volumes of sports-related da

Business Model Canvas:

1. Customer Segments:
- Individuals with mobility limitations, including the elderly and those with disabilities.
- Healthcare institutions such as hospitals, rehabilitation centers, and ambulatory surgical centers.
- Homecare providers and caregivers.

2. Value Propositions:
- Innovative and technologically advanced wheelchairs that enhance user comfort, mobility, and independence.
- Customization options to meet specific user needs and preferences.
- Integration of smart technologies for improved control, connectivity, and obstacle avoidance.
- Sustainable and eco-friendly wheelchair design.
- Ergonomic considerations and enhanced user comfort.

3. Channels:
- Direct sales through partnerships with healthcare institutions, homecare providers, and wheelchair retailers.
- Online sales platforms and e-commerce websites.
- Collaboration with assistive technology organizations and disability advocacy groups for outreach and awareness.

4. Customer Relationships:
- 

In [29]:
    # Extract the response text
    response_text = completion.choices[0].message.content

    # Split the response text into lines for processing
    lines = response_text.split('\n')

    # Process and print each line with additional formatting
    for line in lines:
        # Add an empty line before each numbered point to separate paragraphs
        if line.startswith(tuple(map(str, range(1, 10)))):
            print("\n" + line)
        else:
            print(line)

    # Print a separator line for clarity between different documents
    print("\n" + "="*50 + "\n")


Based on the provided information, here is a business model canvas for Himax Technologies, Inc.:

Key Partners:
- Geely Auto: Himax Technologies collaborates with Geely Auto for the mass production of NEVs (New Energy Vehicles) using its LTDI (Low Temperature Polysilicon Thin-Film Transistor with Integrated Display Driver) technology.
- DESMAN: Himax Technologies collaborates with DESMAN, a smart door lock leader in China, for the WiseEye2 collaboration, enabling the world's first smart door lock with uninterrupted surveillance.

Key Activities:
- Research and Development: Himax Technologies invests in R&D to develop display drivers and other semiconductor products, focusing on automotive, WiseEye AI, and optical technologies.
- Mass Production: Himax Technologies initiates mass production for Geely Auto's NEVs and plans to mass produce touch controller ICs for in-cell TDDI (Touch and Display Driver Integration) in LCD notebooks and NB OLEDs.

Key Resources:
- Automotive Product Portfo